In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
from google.protobuf.json_format import MessageToDict

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [ ]:
cap = cv2.VideoCapture(0)

# Curl counter variables
lcounter = 0 
rcounter=0
lstage = None
rstage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        imgage = cv2.flip(image, 1)
      
        # Make detection
        results = pose.process(image)

        mpHands = mp.solutions.hands
        hands = mpHands.Hands(static_image_mode=False, model_complexity=1,min_detection_confidence=0.75,min_tracking_confidence=0.75, max_num_hands=2)
        hresults = hands.process(image)
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            for i in hresults.multi_handedness:
                
                label = MessageToDict(i)['classification'][0]['label']
  
                if label == 'Left':
                    cv2.putText(image, label+' Hand', (20, 50),cv2.FONT_HERSHEY_COMPLEX, 0.9,(0, 255, 0), 2)
                    lshoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    lelbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    lwrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    langle = calculate_angle(lshoulder, lelbow, lwrist) 
                    cv2.putText(image, str(langle), tuple(np.multiply(lelbow, [640, 480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                    if langle > 160:
                        lstage = "down"
                    if langle < 30 and lstage =='down':
                        lstage="up"
                        lcounter +=1
                        print('left hand',lcounter)
                    cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
                    cv2.putText(image, ' left REPS', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image, str(lcounter), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
                    cv2.putText(image, ' left STAGE', (65,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image, lstage, (60,60),cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))              
                    # Rep data
                   

                if label == 'Right':
                    cv2.putText(image, label+' Hand', (460, 50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (0, 255, 0), 2)
                    rshoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    relbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    rwrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    rangle = calculate_angle(rshoulder, relbow, rwrist) 
                    cv2.putText(image, str(langle), tuple(np.multiply(relbow, [640, 480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                    if rangle > 160:
                        rstage = "down"
                    if rangle < 30 and rstage =='down':
                        rstage="up"
                        rcounter +=1
                        print('right hand',rcounter)
                    cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
                    cv2.putText(image, ' right REPS', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image, str(rcounter), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
                    cv2.putText(image, ' right STAGE', (65,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image, rstage, (60,60),cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 
                    # Rep data
                    
        except:
            pass
          
                    
         
       
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()